In [1]:
import plotly.graph_objects as go

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import numpy as np
import pandas as pd
from pykrx import stock
from pykrx import bond
from time import sleep

from datetime import datetime
from datetime import timedelta
import os
import time
from plotly.subplots import make_subplots
import glob

### 데이터 수집

In [67]:
ticker_nm = '005930'
ticker_nm = '247540'
start_date  = '20190101'
today_date1 = '20231017'

df_raw = stock.get_market_ohlcv(start_date, today_date1, ticker_nm)
df_raw = df_raw.reset_index()
df_raw['ticker'] = ticker_nm

df_raw.columns = ['date', 'open', 'high', 'low', 'close', 'volume','price_change_percentage', 'ticker']

### 이동평균선 추가

In [68]:
df_raw['MA5'] = df_raw['close'].rolling(window=5).mean()
df_raw['MA20'] = df_raw['close'].rolling(window=20).mean()
df_raw['MA60'] = df_raw['close'].rolling(window=60).mean()
df_raw['MA120'] = df_raw['close'].rolling(window=120).mean()

### 볼린져 밴드 계산

In [69]:
std = df_raw['close'].rolling(20).std(ddof=0)

df_raw['upper'] = df_raw['MA20'] + 2 * std
df_raw['lower'] = df_raw['MA20'] - 2 * std

### 볼린져밴드 매수매도타이밍

In [70]:
now = datetime.now()
now = now + timedelta(days=-300)
today_date2 = now.strftime('%Y-%m-%d')
# df_raw = df_raw[df_raw['date'] > today_date2]
df_raw = df_raw[df_raw['date'] > '2023-01-01']
df_raw = df_raw.reset_index(drop = True)


- 밴드 하단을 상향 돌파 했을 경우 매수
- 밴드 상단을 하향 돌파 했을 경우 매도

In [98]:
down_reg_sq = df_raw['upper'] - df_raw['close'] 
top_reg_sq  = df_raw['lower'] - df_raw['close'] 

down_reg = [idx for idx in range(1,len(df_raw)) if down_reg_sq[idx] > 0 and down_reg_sq[idx-1] <= 0]
top_reg = [idx for idx in range(1,len(df_raw)) if top_reg_sq[idx] < 0 and top_reg_sq[idx-1] >= 0]

down_reg_df = pd.DataFrame({
    'index':down_reg,
    'name':'하향 회귀'})

top_reg_df = pd.DataFrame({
    'index':top_reg,
    'name':'상향회귀'})
    
cross_df = pd.concat([down_reg_df, top_reg_df])
cross_df = cross_df.reset_index(drop = True)

0      -1317.136137
1      -1739.116587
2      -3306.758134
3      -2257.344392
4      -4474.543663
           ...     
189   -20988.785983
190   -32500.000000
191   -25215.868648
192   -23143.519964
193   -30642.456729
Length: 194, dtype: float64

### 차트

In [105]:
# fig = go.Figure()
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.01, row_heights=[0.7, 0.3])

# 캔들스틱차트
fig.add_trace(go.Candlestick(
    x=df_raw['date'],
    open=df_raw['open'],
    high=df_raw['high'],
    low=df_raw['low'],
    close=df_raw['close'],
    increasing_line_color= 'red', decreasing_line_color= 'blue', 
    name = ''), row = 1, col = 1)

# MA 5 
fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA5'],
                         opacity=0.7,
                         line=dict(color='blue', width=2),
                         name='MA 5') , row = 1, col = 1)

# MA 20
fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA20'],
                         opacity=0.7,
                         line=dict(color='orange', width=2),
                         name='MA 20'), row = 1, col = 1)

# MA 60
fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA60'],
                         opacity=0.7,
                         line=dict(color='purple', width=2),
                         name='MA 60'), row = 1, col = 1)

# MA 120
fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA120'],
                         opacity=0.7,
                         line=dict(color='green', width=2),
                         name='MA 120'), row = 1, col = 1)


fig.add_trace(go.Scatter(
    x=pd.concat([df_raw['date'], df_raw['date'][::-1]]),
    y=pd.concat([df_raw['upper'], df_raw['lower'][::-1]]),
    fill='toself',
    fillcolor='rgba(255,255,0,0.1)',
    line=dict(color='rgba(255,255,255,0.2)', width=2),
    name='Bollinger Band',
    showlegend=False
), row = 1, col = 1)


# 상향, 하향 회귀
for i in range(len(cross_df)):
    cross_index = cross_df['index'][i]
    cross_name = cross_df['name'][i]
    
    cross_date = df_raw['date'][cross_index]
    cross_value = df_raw['close'][cross_index]

    fig.add_annotation(x=cross_date, y=cross_value,
        text=cross_name,
        showarrow=True,
        arrowhead=1)
    

# Row 2 
# volume
colors = ['blue' if row['open'] - row['close'] >= 0 
          else 'red' for index, row in df_raw.iterrows()]

fig.add_trace(go.Bar(x=df_raw['date'], 
                     y=df_raw['volume'],
                     marker_color=colors,
                     name = 'Volume',
                     showlegend=False
                    ), row=2, col=1)

    
    
# Rayout
fig.update_layout(
    title = '삼성전자 주가',
    title_font_family="맑은고딕",
    title_font_size = 18,
    hoverlabel=dict(
        bgcolor='black',
        font_size=15,
    ),
    hovermode="x unified",
    template='plotly_dark',
    xaxis_tickangle=90,
    yaxis_tickformat = ',',
    legend = dict(orientation = 'h', xanchor = "center", x = 0.5, y= 1.2),
    barmode='group',
    margin=go.layout.Margin(
        l=10, #left margin
        r=10, #right margin
        b=10, #bottom margin
        t=100  #top margin
    ),
    height=400, width=900, 
#     showlegend=False, 
    xaxis_rangeslider_visible=False
)


# update y-axis label
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Volume", row=2, col=1)

fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
fig.show()

/home/shjj08choi/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [106]:
def bolinger_bands(df_raw):
    # fig = go.Figure()
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.01, row_heights=[0.7, 0.3])

    # 캔들스틱차트
    fig.add_trace(go.Candlestick(
        x=df_raw['date'],
        open=df_raw['open'],
        high=df_raw['high'],
        low=df_raw['low'],
        close=df_raw['close'],
        increasing_line_color= 'red', decreasing_line_color= 'blue', 
        name = ''), row = 1, col = 1)

    # MA 5 
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA5'],
                             opacity=0.7,
                             line=dict(color='blue', width=2),
                             name='MA 5') , row = 1, col = 1)

    # MA 20
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA20'],
                             opacity=0.7,
                             line=dict(color='orange', width=2),
                             name='MA 20'), row = 1, col = 1)

    # MA 60
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA60'],
                             opacity=0.7,
                             line=dict(color='purple', width=2),
                             name='MA 60'), row = 1, col = 1)

    # MA 120
    fig.add_trace(go.Scatter(x=df_raw['date'],y=df_raw['MA120'],
                             opacity=0.7,
                             line=dict(color='green', width=2),
                             name='MA 120'), row = 1, col = 1)


    fig.add_trace(go.Scatter(
        x=pd.concat([df_raw['date'], df_raw['date'][::-1]]),
        y=pd.concat([df_raw['upper'], df_raw['lower'][::-1]]),
        fill='toself',
        fillcolor='rgba(255,255,0,0.1)',
        line=dict(color='rgba(255,255,255,0.2)', width=2),
        name='Bollinger Band',
        showlegend=False
    ), row = 1, col = 1)


    # 상향, 하향 회귀
    for i in range(len(cross_df)):
        cross_index = cross_df['index'][i]
        cross_name = cross_df['name'][i]

        cross_date = df_raw['date'][cross_index]
        cross_value = df_raw['close'][cross_index]

        fig.add_annotation(x=cross_date, y=cross_value,
            text=cross_name,
            showarrow=True,
            arrowhead=1)


    # Row 2 
    # volume
    colors = ['blue' if row['open'] - row['close'] >= 0 
              else 'red' for index, row in df_raw.iterrows()]

    fig.add_trace(go.Bar(x=df_raw['date'], 
                         y=df_raw['volume'],
                         marker_color=colors,
                         name = 'Volume',
                         showlegend=False
                        ), row=2, col=1)



    # Rayout
    fig.update_layout(
        title = '삼성전자 주가',
        title_font_family="맑은고딕",
        title_font_size = 18,
        hoverlabel=dict(
            bgcolor='black',
            font_size=15,
        ),
        hovermode="x unified",
        template='plotly_dark',
        xaxis_tickangle=90,
        yaxis_tickformat = ',',
        legend = dict(orientation = 'h', xanchor = "center", x = 0.5, y= 1.2),
        barmode='group',
        margin=go.layout.Margin(
            l=10, #left margin
            r=10, #right margin
            b=10, #bottom margin
            t=100  #top margin
        ),
        height=400, width=900, 
    #     showlegend=False, 
        xaxis_rangeslider_visible=False
    )


    # update y-axis label
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)

    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
    return fig

In [107]:
 bolinger_bands(df_raw)

/home/shjj08choi/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

